In [1]:
# cd.. In separate cell if used

from lib.network_mapper import network_mapper_function
from lib.NR3 import NR3_function
from lib.generatenoise import *
#from lib.computeFTreal import computeFTrealfunction
#from lib.computeJTreal import computeJTrealfunction
#from lib import *
import opendssdirect as dss

import re
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import random
#matplotlib inline
fig_size = np.array([18, 6])
plt.rcParams["figure.figsize"] = fig_size

# z = exp(s*T)

# zoh
# z ~ 1 + s*T
# s ~ (z - 1)/T


# bilinear transform
# z ~ (2 + sT)/(2 - sT)
# s ~ (2/T)(z - 1)/(z + 1)

In [4]:
fp = "C:\\Users\\kathl\\Desktop\\LinDist3Flow\\20180601\\NETWORKS\\"

fn = 'compare__lindist3flow_05node_threephase_unbalanced_oscillation_03.txt'
#fn = '05node_singlephase_radial.txt'
# #fn = "03node_singlephase_radial_example.txt"

# fn = "03node_fullphase_radial_example.txt" #works
# fn = '03node_fullphase_mesh_example.txt' #works, refers to spu nom
#fn = 'ieee_13node_mesh_open.txt' #works
#fn = '05node_fullphase_radial.txt'; #works

#fn = 'ieee_13node_balance.txt' #works 
# fn = 'ieee_13node_mesh_open.txt'; #works
#fn = 'ieee_34node.txt' #works
#fn = 'ieee_37node.txt'

network1 = network_mapper_function(fp, fn)

nnode1 = network1.nodes.nnode
nline1 = network1.lines.nline

# Network paramaters

nnode = network1.nodes.nnode
nline = network1.lines.nline

# Load parameters

network1.loads.aPQ = .750*np.ones((3,nnode1))*(network1.loads.spu != 0)
network1.loads.aI = 0.10*np.ones((3,nnode1))*(network1.loads.spu != 0)
network1.loads.aZ = 0.15*np.ones((3,nnode1))*(network1.loads.spu != 0)


network1.loads.spu = 1.25*network1.loads.spu


# Capacitor parameters

#network1.caps.cappu = 1*network1.caps.cappu

# Controller parameters

#network1.cons.wmaxpu = 0.5*network1.cons.wmaxpu


BASE
Vbase: 2771.2799999999997
Sbase: 1000000.0
Ibase: 360.8440864871107
Zbase: 7.679992838399998

NODES
nnode: 7
nodelist: ['sub', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06']
phases: ['abc', 'abc', 'abc', 'abc', 'ac', 'ab', 'b']
PH:
 [[1 1 1 1 1 1 0]
 [1 1 1 1 0 1 1]
 [1 1 1 1 1 0 0]]

LINES
nline: 6
TXnode: ['sub', 'A01', 'A02', 'A03', 'A03', 'A03']
RXnode: ['A01', 'A02', 'A03', 'A04', 'A05', 'A05']
TXnum: [0 1 2 3 3 3]
RXnum: [1 2 3 4 5 5]
phases: ['abc', 'abc', 'abc', 'ac', 'ab', 'b']
PH:
 [[1 1 1 1 1 0]
 [1 1 1 0 1 1]
 [1 1 1 1 0 0]]
config: ['sub_a01', 'a01_a02', 'a02_a03', 'a03_a04', 'a03_a05', 'a05_a06']
length: [ 1.  1.  1.  1.  1.  1.]

NODES + LINES
inlines:
 [[-1  0  1  2  3  4 -1]
 [-1 -1 -1 -1 -1  5 -1]]
innodes:
 [[-1  0  1  2  3  3 -1]
 [-1 -1 -1 -1 -1  3 -1]]
outlines:
 [[ 0  1  2  3 -1 -1 -1]
 [-1 -1 -1  4 -1 -1 -1]
 [-1 -1 -1  5 -1 -1 -1]]
outnodes:
 [[ 1  2  3  4 -1 -1 -1]
 [-1 -1 -1  5 -1 -1 -1]
 [-1 -1 -1  5 -1 -1 -1]]
[[ 0.015+0.01j  0.005+0.04j  0.005+0.04j]
 [ 0.

ValueError: 'a02' is not in list

In [ ]:
network1.cons.wpu = np.zeros((3,nnode))
network1.vvc.vvcpu = np.zeros((3,nnode))

slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])


VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
    NR3_function(network1,slackidx,Vslack,None,None,None,None)

'''
VNR = np.round(VNR,decimals=6)
INR = np.round(INR,decimals=6)
STXNR = np.round(STXNR,decimals=6)
SRXNR = np.round(SRXNR,decimals=6)
iNR = np.round(iNR,decimals=6)
sNR = np.round(sNR,decimals=6)
'''

print('VNR\n', np.round(VNR,decimals=6))
print('INR\n', np.round(INR,decimals=6))
print('STXNR\n', np.round(STXNR,decimals=6))
print('SRXNR\n', np.round(SRXNR,decimals=6))
print('iNR\n', np.round(iNR,decimals=6))
print('sNR\n', np.round(sNR,decimals=6))
print('itercount:', itercount)

print('|VNR|\n', np.round(np.abs(VNR),decimals=6))
print('<VNR\n', np.round(180/np.pi*np.angle(VNR),decimals=6))
print('D<VNR\n', 180/np.pi*np.angle(VNR) - 180/np.pi*np.angle(VNR[:,[0]]))

In [ ]:
fid = open('NR3-' + fn[:-4] + '-no-python.txt','w')

phstr = ['a','b','c']

fid.write('VNR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nnode):
        wstr = '{0:.6f}'.format(VNR[ph,k1].real) + ' ' + '{0:.6f}'.format(VNR[ph,k1].imag) + '\n'
        fid.write(wstr)

fid.write('\n\nINR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nline):
        wstr = '{0:.6f}'.format(INR[ph,k1].real) + ' ' + '{0:.6f}'.format(INR[ph,k1].imag) + '\n'
        fid.write(wstr)
        
fid.write('\n\nSTXNR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nline):
        wstr = '{0:.6f}'.format(STXNR[ph,k1].real) + ' ' + '{0:.6f}'.format(STXNR[ph,k1].imag) + '\n'
        fid.write(wstr)

fid.write('\n\nSRXNR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nline):
        wstr = '{0:.6f}'.format(SRXNR[ph,k1].real) + ' ' + '{0:.6f}'.format(SRXNR[ph,k1].imag) + '\n'
        fid.write(wstr)
        
fid.write('\n\niNR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nnode):
        wstr = '{0:.6f}'.format(iNR[ph,k1].real) + ' ' + '{0:.6f}'.format(iNR[ph,k1].imag) + '\n'
        fid.write(wstr)
        
fid.write('\n\nsNR\n\n')
for ph in range(0,3):
    wstr = phstr[ph] + '\n'
    fid.write(wstr)
    for k1 in range(0,nnode):
        wstr = '{0:.6f}'.format(sNR[ph,k1].real) + ' ' + '{0:.6f}'.format(sNR[ph,k1].imag) + '\n'
        fid.write(wstr)

fid.close()

In [ ]:
#Count the number of phases that exist
phase_count = 0;
a = 0;
b = 0;
c = 0;
for phaseCount in range(len(network1.nodes.phases)):
    curr = network1.nodes.phases[phaseCount];
    if 'a' in curr and a == 0:
        phase_count = phase_count + 1
        a = 1
    if 'b' in curr and b == 0:
        b = 1
        phase_count = phase_count + 1
    if 'c' in curr and c == 0:
        phase_count = phase_count+1
        c = 1

        
csv_file_count = 1;
csvfile = pd.read_csv(r'C:\Users\kathl\Desktop\LinDist3Flow\20180601\testpvnum1\node_1_pv_1_minute_normalized.csv')    
profiles = np.zeros((3, nnode, len(csvfile)))
#Assign each node and phase a loading profile via all_demand
for ph in range(3):
    for kn in range(nnode): 
        if (a == 1 and ph == 0) or (b == 1 and ph == 1) or (c == 1 and ph == 2):
            csvfile = pd.read_csv(r'C:\Users\kathl\Desktop\LinDist3Flow\20180601\testpvnum' + \
                                  str(csv_file_count // 38)+'\\node_'+ str((csv_file_count % 37) + 1) + \
                                  '_pv_'+str(csv_file_count // 38)+'_minute_normalized.csv')
            profiles[ph,kn] = csvfile.iloc[:, 5].values
            csv_file_count = csv_file_count + 1          

In [ ]:
ts = 20

In [ ]:
pQ_noise, Z_noise, I_noise = generate_pQ_z_I_noise(ts, nnode)

In [ ]:
fid = open('NR3-python.txt','w')

# Compute a spu nominal based on the node and phase's assigned loading profile
for kt in range(ts): #time steps
    for ph in range(3): #3 phases
        for kn in range(nnode): #nnodes
            if (a == 1 and ph == 0) or (b == 1 and ph == 1) or (c == 1 and ph == 2):
                network1.loads.spu_nominal[ph,kn] = network1.loads.spu[ph,kn] * profiles[ph,kn,kt] * generate_spu_noise()
        
    network1 = add_noise(network1, kt, nnode, pQ_noise, Z_noise, I_noise)
    
    network1.cons.wpu = np.zeros((3,nnode))
    network1.vvc.vvcpu = np.zeros((3,nnode))

    slackidx = 0
    Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])
    
    VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
        NR3_function(network1,slackidx,Vslack,None,None,None,None)

    phstr = ['a','b','c']
    
    fid = open('NR3-python.txt','a')
 
    fid.write('VNR\n\n')
    for ph in range(0,3):
        wstr = phstr[ph] + '\n'
        fid.write(wstr)
        for k1 in range(0,nnode):
            wstr = '{0:.6f}'.format(VNR[ph,k1].real) + ' ' + '{0:.6f}'.format(VNR[ph,k1].imag) + '\n'
            fid.write(wstr)

    fid.write('\n\nINR\n\n')
    for ph in range(0,3):
        wstr = phstr[ph] + '\n'
        fid.write(wstr)
        for k1 in range(0,nline):
            wstr = '{0:.6f}'.format(INR[ph,k1].real) + ' ' + '{0:.6f}'.format(INR[ph,k1].imag) + '\n'
            fid.write(wstr)

    fid.write('\n\nSTXNR\n\n')
    for ph in range(0,3):
        wstr = phstr[ph] + '\n'
        fid.write(wstr)
        for k1 in range(0,nline):
            wstr = '{0:.6f}'.format(STXNR[ph,k1].real) + ' ' + '{0:.6f}'.format(STXNR[ph,k1].imag) + '\n'
            fid.write(wstr)

    fid.write('\n\nSRXNR\n\n')
    for ph in range(0,3):
        wstr = phstr[ph] + '\n'
        fid.write(wstr)
        for k1 in range(0,nline):
            wstr = '{0:.6f}'.format(SRXNR[ph,k1].real) + ' ' + '{0:.6f}'.format(SRXNR[ph,k1].imag) + '\n'
            fid.write(wstr)

    fid.write('\n\niNR\n\n')
    for ph in range(0,3):
        wstr = phstr[ph] + '\n'
        fid.write(wstr)
        for k1 in range(0,nnode):
            wstr = '{0:.6f}'.format(iNR[ph,k1].real) + ' ' + '{0:.6f}'.format(iNR[ph,k1].imag) + '\n'
            fid.write(wstr)

    fid.write('\n\nsNR\n\n')
    for ph in range(0,3):
        wstr = phstr[ph] + '\n'
        fid.write(wstr)
        for k1 in range(0,nnode):
            wstr = '{0:.6f}'.format(sNR[ph,k1].real) + ' ' + '{0:.6f}'.format(sNR[ph,k1].imag) + '\n'
            fid.write(wstr)

    fid.close()